# Import library

In [4]:
import pandas as pd
import transformers
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
from datasets import Dataset

# Cek GPU

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

print(transformers.__version__)

Using device: cuda
4.40.1


# Data

In [6]:
data = {
    'text': [
        'saya sangat menikmati film ini, ceritanya menarik banget',
        'akting pemain film ini luar biasa, sangat menghibur',
        'film ini memiliki plot yang luar biasa dan sangat menyentuh',
        'film yang sangat cocok untuk ditonton bersama keluarga',
        'saya sangat terkesan dengan akting aktor utama film ini',
        'film ini tidak sesuai dengan harapan saya, sangat membosankan',
        'alur film ini sangat datar dan kurang menarik',
        'saya kecewa dengan kualitas film ini, tidak sebanding dengan review',
        'film ini terlalu panjang dan membosankan, tidak ada yang istimewa',
        'cerita film ini sangat kaku dan tidak masuk akal',
        'musik dalam film ini sangat menyentuh dan menambah suasana',
        'pengambilan gambar dalam film ini sangat estetis dan menawan',
        'skenario film ini sangat kuat dan penuh kejutan',
        'film ini sangat buruk, banyak adegan yang tidak jelas',
        'saya bingung dengan alur cerita film ini, tidak konsisten'
    ],
    'label': [
        'positif', 'positif', 'positif', 'positif', 'positif',
        'negatif', 'negatif', 'negatif', 'negatif', 'negatif',
        'positif', 'positif', 'positif', 'negatif', 'negatif'
    ]
}

df = pd.DataFrame(data)

# Label encoding

In [7]:
df['label'] = df['label'].map({'negatif': 0, 'positif': 1})

# Split data

In [8]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)

# Load IndoBert tokenizer

In [9]:
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")

c:\Users\Abiyoso\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Tokenization

In [10]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


# Convert ke Dataset

In [11]:
train_dataset = Dataset.from_dict({**train_encodings, 'label': train_labels.tolist()})
test_dataset = Dataset.from_dict({**test_encodings, 'label': test_labels.tolist()})

# Load IndoBERT untuk classification

In [12]:
model = BertForSequenceClassification.from_pretrained("indobenchmark/indobert-base-p1", num_labels=2).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Training Arguments

In [13]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    seed=42,
)

# Trainer

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
)

# Train Model

In [15]:
trainer.train()

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.6118221879005432, 'eval_runtime': 0.0798, 'eval_samples_per_second': 37.584, 'eval_steps_per_second': 12.528, 'epoch': 1.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.4353550374507904, 'eval_runtime': 0.0674, 'eval_samples_per_second': 44.478, 'eval_steps_per_second': 14.826, 'epoch': 2.0}


...

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.26879024505615234, 'eval_runtime': 0.0756, 'eval_samples_per_second': 39.67, 'eval_steps_per_second': 13.223, 'epoch': 4.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.22607390582561493, 'eval_runtime': 0.0728, 'eval_samples_per_second': 41.198, 'eval_steps_per_second': 13.733, 'epoch': 5.0}
{'train_runtime': 1.9616, 'train_samples_per_second': 30.588, 'train_steps_per_second': 7.647, 'train_loss': 0.3609752058982849, 'epoch': 5.0}


TrainOutput(global_step=15, training_loss=0.3609752058982849, metrics={'train_runtime': 1.9616, 'train_samples_per_second': 30.588, 'train_steps_per_second': 7.647, 'total_flos': 400833248400.0, 'train_loss': 0.3609752058982849, 'epoch': 5.0})

# Evaluate

In [16]:
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)

print("\nClassification Report:")
print(classification_report(test_labels, preds, target_names=["negatif", "positif"]))

  0%|          | 0/1 [00:00<?, ?it/s]


Classification Report:
              precision    recall  f1-score   support

     negatif       1.00      1.00      1.00         1
     positif       1.00      1.00      1.00         2

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3

